### Imports ###

In [1]:
import pandas as pd
import regex as re
from datetime import datetime
import time

### Read in Tweepy Scrape ###

In [2]:
df = pd.read_csv('first_scrape.csv')

In [3]:
df.head()

,Tweet Text,Author,Coordinates,Hashtags (Location),Creation Time (UTC)
0,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],2019-07-29 16:23:49
1,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],2019-07-29 16:12:13
2,A crash is blocking the three left lanes. in #...,TotalTrafficLA,"[34.06873, -117.8612]",['WestCovina'],2019-07-29 15:54:37
3,A motorcycle crash was moved to the right shou...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],2019-07-29 15:19:49
4,A motorcycle crash is blocking the left lane. ...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],2019-07-29 14:53:08


In [4]:
df.shape

(63, 5)

### Remove Tweets Involving Re-opens ###

In [5]:
def bad_tweet(tweet_text):
    delete = 0
    for i in ['clear', 'reopen']:
        if i in tweet_text:
            delete=1
    if delete:
        return True
    else:
        return False

In [6]:
df['bad tweet'] = df['Tweet Text'].apply(bad_tweet)

In [7]:
df = df[df['bad tweet']==False]

In [8]:
df.drop(labels=['bad tweet'],axis=1,inplace=True)

In [9]:
df.head()

,Tweet Text,Author,Coordinates,Hashtags (Location),Creation Time (UTC)
0,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],2019-07-29 16:23:49
1,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],2019-07-29 16:12:13
2,A crash is blocking the three left lanes. in #...,TotalTrafficLA,"[34.06873, -117.8612]",['WestCovina'],2019-07-29 15:54:37
3,A motorcycle crash was moved to the right shou...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],2019-07-29 15:19:49
4,A motorcycle crash is blocking the left lane. ...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],2019-07-29 14:53:08


In [10]:
df.shape

(57, 5)

### Functions to Parse Tweets ###

In [11]:
#Function to identify what road is closed
def find_road_improved(text):
    #Split string on the word on
    on_split = text.split(' on ') 
    if len(on_split) == 1:
        return None
    else:
        #return the first 2 words after the word on
        output = " ".join(on_split[1].split(" ")[0:2])
        #check if the first character of the 2 word string is a number or a capital I, H, U
        if bool(re.match(r'[HIU0-9]',output)):
            return output

In [12]:
#Function to identify the starting point of the road closure
def find_start_improved(text):
    #Split string on the word on, the starting point of the road closure should be after this
    on_split = text.split(' on ')
    if len(on_split) == 1:
        return None
    else:
        #Generally the starting point of the road closure is after the word "on", before the comma and after one
        #of the following words
        comma_split = on_split[1].split(",")[0]
        if len(comma_split.split(" at ")) > 1:
            return comma_split.split(" at ")[1].split(" and")[0].split(" #")[0]
        elif len(comma_split.split(" before ")) > 1:
            return comma_split.split(" before ")[1].split(" and")[0].split(" #")[0]
        elif len(comma_split.split(" after ")) > 1:
            return comma_split.split(" after ")[1].split(" and")[0].split(" #")[0]
        elif len(comma_split.split(" between ")) > 1:
            return comma_split.split(" between ")[1].split(" and")[0].split(" #")[0]
        elif len(comma_split.split(" approaching ")) > 1:
            return comma_split.split(" approaching ")[1].split(" and")[0].split(" #")[0]

In [13]:
#Function to identify the ending point of the road closure
def find_end_improved(text):
    #Split string on the word on, the starting point of the road closure should be after this
    on_split = text.split(' on ')
    if len(on_split) == 1:
        return None
    else:
        #Generally the ending point of the road closure is after the word "on" and after either the word "to" or "and"
        after_on = on_split[1]
        if len(after_on.split(" to ")) > 1:
            return after_on.split(" to ")[1].split(",")[0].split(" #")[0]
        elif len(after_on.split(" and ")) > 1:
            return after_on.split(" and ")[1].split(",")[0].split(" #")[0]

In [14]:
#Function to convert UTC timestamp to local timestamp
#Function obtained from this stackoverflow post
#https://stackoverflow.com/questions/4770297/convert-utc-datetime-string-to-local-datetime

def datetime_from_utc_to_local(utc_datetime):
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

### Apply functions to dataframe ###

In [16]:
df['Closed Road'] = df['Tweet Text'].apply(find_road_improved)

In [17]:
df['Start Point Of Closure'] = df['Tweet Text'].apply(find_start_improved)

In [18]:
df['End Point Of Closure'] = df['Tweet Text'].apply(find_end_improved)

In [19]:
df['date'] = pd.to_datetime(df['Creation Time (UTC)'])
df['Creation Time'] = df['date'].apply(datetime_from_utc_to_local)

In [20]:
df.drop(labels=['Creation Time (UTC)','date'],axis=1,inplace=True)

In [21]:
df.head()

,Tweet Text,Author,Coordinates,Hashtags (Location),Closed Road,Start Point Of Closure,End Point Of Closure,Creation Time
0,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],210 EB,Michillinda Ave,Sierra Madre,2019-07-29 09:23:49
1,A crash is blocking the carpool lane. in #Arca...,TotalTrafficLA,"[34.14892, -118.0632]",['Arcadia'],210 EB,Michillinda Ave,Madre St,2019-07-29 09:12:13
2,A crash is blocking the three left lanes. in #...,TotalTrafficLA,"[34.06873, -117.8612]",['WestCovina'],I-10 WB,Holt Ave,Via Verde,2019-07-29 08:54:37
3,A motorcycle crash was moved to the right shou...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],405 NB,La Tijera Blvd,Century Blvd,2019-07-29 08:19:49
4,A motorcycle crash is blocking the left lane. ...,TotalTrafficLA,"[33.97027, -118.3747]",['Westchester'],405 NB,La Tijera Blvd,Century Blvd,2019-07-29 07:53:08


In [22]:
#Above functions can return a None of if they don't find appropriate data, dropping those rows
df.dropna(inplace=True)

In [23]:
#Remove duplicate data
df.drop_duplicates(subset='Coordinates',inplace=True)

In [24]:
df.shape

(35, 8)

In [27]:
#Output to CSV
df.to_csv('Twitter_Output_For_Dataframe.csv')